# Complete Workflow: Roll Rate Model - Từ Data đến Report

Notebook này chạy toàn bộ workflow:
1. Load & prepare data
2. Build transition matrices
3. Forecast lifecycle
4. Calibration (k per MOB)
5. Apply calibration & aggregate
6. Allocate xuống loan-level (MOB 12 & 24) + Chi tiết hợp đồng
7. Analysis & visualization
8. Export reports

**Thời gian chạy:** ~5-10 phút (tùy data size)

In [ ]:
# Setup path
import sys
from pathlib import Path

project_root = Path(".").resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"📁 Project root: {project_root}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Config
from src.config import CFG, BUCKETS_CANON, BUCKETS_30P, BUCKETS_90P

# Data loader
from src.data_loader import load_data

# Transition
from src.rollrate.transition import compute_transition_by_mob

# Lifecycle & Forecast
from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    build_full_lifecycle_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    aggregate_products_to_portfolio,
    export_lifecycle_all_products_one_file,
    extend_actual_info_with_portfolio,
)

# Calibration
from src.rollrate.calibration_kmob import (
    fit_k_raw,
    smooth_k,
    fit_alpha,
    forecast_all_vintages_partial_step,
    backtest_error_by_mob,
)

# Allocation
from src.rollrate.allocation_multi_mob import (
    allocate_multi_mob_with_del_metrics,
    compare_del_across_mobs,
    export_multi_mob_to_excel,
    pivot_del_by_product_mob,
)

print("✅ Import thành công")

---
## 1️⃣ LOAD & PREPARE DATA

In [ ]:
# Load data
DATA_PATH = 'C:/Users/User/Projection_PB/Projection_pb/ETB_Parquet_YYYYMM'  # 🔥 Thay đổi path của bạn

df_raw = load_data(DATA_PATH)

# Chuẩn hóa - dùng parse_date_column để hỗ trợ cả YYYYMM và datetime
from src.config import parse_date_column, create_segment_columns, SEGMENT_COLS
df_raw['DISBURSAL_DATE'] = parse_date_column(df_raw['DISBURSAL_DATE'])

# Tạo PRODUCT_TYPE và RISK_SCORE từ SEGMENT_COLS trong config
# Nếu SEGMENT_COLS có nhiều cột (vd: RISK_SCORE, GENDER, LA_GROUP)
# => RISK_SCORE sẽ được ghép thành 1 cột duy nhất
df_raw = create_segment_columns(df_raw)

print(f"\n📊 Data Summary:")
print(f"   SEGMENT_COLS: {SEGMENT_COLS}")
print(f"   Total rows: {len(df_raw):,}")
print(f"   Date range: {df_raw[CFG['cutoff']].min()} → {df_raw[CFG['cutoff']].max()}")
print(f"   Products: {df_raw['PRODUCT_TYPE'].nunique()}")
print(f"   Risk scores: {df_raw['RISK_SCORE'].nunique()}")
print(f"   Loans: {df_raw[CFG['loan']].nunique():,}")

In [ ]:
df_raw

---
## 2️⃣ BUILD TRANSITION MATRICES

In [ ]:
print("🔨 Building transition matrices...")
print("   (Có thể mất 1-2 phút)\n")

matrices_by_mob, parent_fallback = compute_transition_by_mob(df_raw)

print(f"\n✅ Hoàn tất:")
print(f"   Products: {len(matrices_by_mob)}")
print(f"   Total matrices: {sum(len(mob_dict) for mob_dict in matrices_by_mob.values())}")

---
## 3️⃣ BUILD LIFECYCLE (ACTUAL + FORECAST)

In [ ]:
print("🔨 Building lifecycle...")

max_mob = 36  # Forecast đến MOB 36

# Build lifecycle (actual + forecast)
df_lifecycle = build_full_lifecycle_amount(
    df_raw=df_raw,
    matrices_by_mob=matrices_by_mob,
    max_mob=max_mob
)

# Tag forecast rows
df_lifecycle = tag_forecast_rows_amount(df_lifecycle, df_raw)

# Add DEL metrics
df_lifecycle = add_del_metrics(df_lifecycle, df_raw)

print(f"\n✅ Lifecycle:")
print(f"   Total rows: {len(df_lifecycle):,}")
print(f"   Actual rows: {(df_lifecycle['IS_FORECAST']==0).sum():,}")
print(f"   Forecast rows: {(df_lifecycle['IS_FORECAST']==1).sum():,}")
print(f"   MOB range: {df_lifecycle['MOB'].min()} → {df_lifecycle['MOB'].max()}")

---
## 4️⃣ CALIBRATION (K PER MOB)

In [ ]:
print("🔨 Calibration: Fitting k per MOB...\n")

# Prepare data
states = BUCKETS_CANON
s30_states = BUCKETS_30P

# Actual results
actual_results = get_actual_all_vintages_amount(df_raw)

# DISB_TOTAL map
loan_disb = (
    df_raw.groupby(["PRODUCT_TYPE", "RISK_SCORE", CFG["orig_date"], CFG["loan"]])[CFG["disb"]]
    .first()
)
cohort_disb = loan_disb.groupby(level=[0, 1, 2]).sum()
disb_total_by_vintage = cohort_disb.to_dict()

print("Step 1: Fit k_raw...")
k_raw_by_mob, weight_by_mob, k_raw_df = fit_k_raw(
    actual_results=actual_results,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    states=states,
    s30_states=s30_states,
    include_co=True,
    denom_mode="disb",
    disb_total_by_vintage=disb_total_by_vintage,
    min_disb=1e-10,
    weight_mode="equal",
    method="wls",
    eps=1e-8,
    min_denom=1e-10,
    min_obs=5,
    fallback_k=1.0,
    return_detail=True,
)

print(f"\n✅ k_raw fitted for {len(k_raw_by_mob)} MOBs")

In [ ]:
print("Step 2: Smooth k...")

if k_raw_by_mob:
    mob_min = min(k_raw_by_mob.keys())
    mob_max = max(k_raw_by_mob.keys())
else:
    mob_min, mob_max = 0, 0

k_smooth_by_mob, mobs, _ = smooth_k(
    k_raw_by_mob=k_raw_by_mob,
    weight_by_mob=weight_by_mob,
    mob_min=mob_min,
    mob_max=mob_max,
    gamma=10.0,
    monotone=False,
    use_cvxpy=True,
    default_k=1.0,
)

print(f"✅ k_smooth computed")

In [ ]:
print("Step 3: Fit alpha (optional)...")

ALPHA_TARGET_MOB = min(max_mob, mob_max) if mob_max else max_mob

alpha, k_final_by_mob, alpha_scores = fit_alpha(
    actual_results=actual_results,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    states=states,
    s30_states=s30_states,
    k_smooth_by_mob=k_smooth_by_mob,
    mob_target=ALPHA_TARGET_MOB,
    include_co=True,
    alpha_grid=None,
    val_frac=0.2,
)

print(f"\n✅ Calibration hoàn tất:")
print(f"   Alpha: {alpha:.4f}")
print(f"   k_final: {len(k_final_by_mob)} MOBs")

In [ ]:
# Visualize k curves
import matplotlib.pyplot as plt

mobs_list = sorted(set(k_raw_by_mob) | set(k_smooth_by_mob) | set(k_final_by_mob))
kr = [k_raw_by_mob.get(m, np.nan) for m in mobs_list]
ks = [k_smooth_by_mob.get(m, np.nan) for m in mobs_list]
kf = [k_final_by_mob.get(m, np.nan) for m in mobs_list]

plt.figure(figsize=(12, 6))
plt.plot(mobs_list, kr, marker="o", label="k_raw", alpha=0.6)
plt.plot(mobs_list, ks, marker="s", label="k_smooth", linewidth=2)
plt.plot(mobs_list, kf, marker="^", label="k_final", linewidth=2)
plt.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label="k=1.0")
plt.xlabel("MOB")
plt.ylabel("k")
plt.title("Calibration k Curves")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 k curves plotted")

---
## 5️⃣ APPLY CALIBRATION & AGGREGATE

In [ ]:
print("🔨 Applying calibration...")

# Forecast với k_final (calibrated)
forecast_calibrated = forecast_all_vintages_partial_step(
    actual_results=actual_results,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    max_mob=max_mob,
    k_by_mob=k_final_by_mob,
    states=states,
)

# Convert to DataFrame
from src.rollrate.lifecycle import (
    lifecycle_to_long_df_amount,
    combine_all_lifecycle_amount,
)

# ✅ FIX: Combine actual + forecast TRƯỚC KHI convert to DataFrame
lifecycle_combined = combine_all_lifecycle_amount(
    actual=actual_results,
    forecast=forecast_calibrated
)

# Convert to long format
df_lifecycle_final = lifecycle_to_long_df_amount(lifecycle_combined)

# Tag forecast rows
df_lifecycle_final = tag_forecast_rows_amount(df_lifecycle_final, df_raw)

# Add DEL metrics
df_lifecycle_final = add_del_metrics(df_lifecycle_final, df_raw)

print(f"\n✅ Lifecycle final (calibrated):")
print(f"   Total rows: {len(df_lifecycle_final):,}")
print(f"   Actual rows: {(df_lifecycle_final['IS_FORECAST']==0).sum():,}")
print(f"   Forecast rows (calibrated): {(df_lifecycle_final['IS_FORECAST']==1).sum():,}")
print(f"   MOB range: {df_lifecycle_final['MOB'].min()} → {df_lifecycle_final['MOB'].max()}")

# ✅ Kiểm tra forecast có data không
df_fc_check = df_lifecycle_final[df_lifecycle_final['IS_FORECAST'] == 1]
if df_fc_check.empty:
    print("\n⚠️ WARNING: Không có forecast rows!")
else:
    print(f"\n✅ Forecast check:")
    print(f"   Forecast MOB range: {df_fc_check['MOB'].min()} → {df_fc_check['MOB'].max()}")
    print(f"   Sample forecast EAD: {df_fc_check[BUCKETS_CANON].sum(axis=1).sum():,.0f}")

In [ ]:
print("🔨 Aggregating to product level...")

# ✅ Kiểm tra trước khi aggregate
print(f"\nBefore aggregate:")
print(f"   Total rows: {len(df_lifecycle_final):,}")
print(f"   Actual: {(df_lifecycle_final['IS_FORECAST']==0).sum():,}")
print(f"   Forecast: {(df_lifecycle_final['IS_FORECAST']==1).sum():,}")

# Aggregate to product
df_product = aggregate_to_product(df_lifecycle_final)

# ✅ Kiểm tra sau aggregate
print(f"\nAfter aggregate to product:")
print(f"   Total rows: {len(df_product):,}")
if 'IS_FORECAST' in df_product.columns:
    print(f"   Actual: {(df_product['IS_FORECAST']==0).sum():,}")
    print(f"   Forecast: {(df_product['IS_FORECAST']==1).sum():,}")

# Aggregate to portfolio
df_portfolio = aggregate_products_to_portfolio(
    df_product,
    portfolio_name="PORTFOLIO_ALL"
)

# Combine
df_del_all = pd.concat([df_product, df_portfolio], ignore_index=True)

print(f"\n✅ Aggregation complete:")
print(f"   Product-level: {len(df_product):,} rows")
print(f"   Portfolio-level: {len(df_portfolio):,} rows")
print(f"   Combined: {len(df_del_all):,} rows")

# ✅ Final check
if 'IS_FORECAST' in df_del_all.columns:
    fc_count = (df_del_all['IS_FORECAST']==1).sum()
    if fc_count == 0:
        print("\n⚠️ WARNING: df_del_all không có forecast rows!")
        print("   Có thể do aggregate_to_product() không preserve IS_FORECAST")
    else:
        print(f"\n✅ df_del_all có {fc_count:,} forecast rows")

---
## 6️⃣ ALLOCATE TO LOAN-LEVEL (MOB 12 & 24)

In [ ]:
print("🔨 Allocating forecast to loan-level...")
print("   Target MOBs: 12, 24")
print("   (Có thể mất 1-2 phút)\n")

df_loan_forecast = allocate_multi_mob_with_del_metrics(
    df_lifecycle_final=df_lifecycle_final,
    df_raw=df_raw,
    target_mobs=[12, 24],
    allocation_method="simple",
    include_del30=True,
    include_del60=False,
    include_del90=True,
)

print(f"\n✅ Loan-level forecast: {len(df_loan_forecast):,} loans")

### 📋 Chi tiết hợp đồng (Loan Details)

Kết quả `df_loan_forecast` đã có sẵn tất cả thông tin chi tiết hợp đồng từ `df_raw`:
- AGREEMENT_ID, CUSTOMER_ID
- PRODUCT_TYPE, RISK_SCORE
- STATE_FORECAST_MOB12, STATE_FORECAST_MOB24
- DEL30_FLAG_MOB12, DEL90_FLAG_MOB12, ...
- Và tất cả các cột khác từ df_raw

In [ ]:
print("📊 Chi tiết hợp đồng sau khi allocate:\n")

# Hiển thị các cột có sẵn
print(f"1️⃣ Tổng số cột: {len(df_loan_forecast.columns)}")
print(f"\n📋 Các cột quan trọng:")
important_cols = [
    'AGREEMENT_ID',
    'CUSTOMER_ID', 
    'PRODUCT_TYPE',
    'RISK_SCORE',
    'STATE_FORECAST_MOB12',
    'STATE_FORECAST_MOB24',
    'DEL30_FLAG_MOB12',
    'DEL90_FLAG_MOB12',
    'DEL30_FLAG_MOB24',
    'DEL90_FLAG_MOB24'
]

# Kiểm tra cột nào có sẵn
available_cols = [col for col in important_cols if col in df_loan_forecast.columns]
for col in available_cols:
    print(f"   ✅ {col}")

# Hiển thị 10 hợp đồng đầu tiên
print(f"\n2️⃣ Sample 10 hợp đồng đầu tiên:")
display_cols = [col for col in available_cols if col in df_loan_forecast.columns]
print(df_loan_forecast[display_cols].head(10))

In [ ]:
# Phân tích chi tiết theo sản phẩm
print("\n3️⃣ Phân tích theo sản phẩm:\n")

# Số lượng hợp đồng theo sản phẩm
product_count = df_loan_forecast.groupby('PRODUCT_TYPE').size().reset_index(name='Count')
print("📊 Số lượng hợp đồng theo sản phẩm:")
print(product_count)

# DEL90 rate theo sản phẩm tại MOB 12
if 'DEL90_FLAG_MOB12' in df_loan_forecast.columns:
    del90_by_product = df_loan_forecast.groupby('PRODUCT_TYPE').agg({
        'DEL90_FLAG_MOB12': ['sum', 'mean']
    }).reset_index()
    del90_by_product.columns = ['PRODUCT_TYPE', 'DEL90_Count', 'DEL90_Rate']
    del90_by_product['DEL90_Rate'] = del90_by_product['DEL90_Rate'] * 100
    
    print("\n📊 DEL90 @ MOB 12 theo sản phẩm:")
    print(del90_by_product)

In [ ]:
# Lọc hợp đồng có rủi ro cao (DEL90 @ MOB 12)
if 'DEL90_FLAG_MOB12' in df_loan_forecast.columns:
    print("\n4️⃣ Hợp đồng có rủi ro cao (DEL90 @ MOB 12):\n")
    
    df_high_risk = df_loan_forecast[df_loan_forecast['DEL90_FLAG_MOB12'] == 1].copy()
    
    print(f"📊 Tổng số hợp đồng DEL90: {len(df_high_risk):,} ({len(df_high_risk)/len(df_loan_forecast)*100:.2f}%)")
    
    # Top 10 hợp đồng rủi ro cao
    if len(df_high_risk) > 0:
        print("\n📋 Top 10 hợp đồng rủi ro cao:")
        risk_cols = ['AGREEMENT_ID', 'PRODUCT_TYPE', 'RISK_SCORE', 
                     'STATE_FORECAST_MOB12', 'DEL90_FLAG_MOB12']
        risk_cols = [col for col in risk_cols if col in df_high_risk.columns]
        print(df_high_risk[risk_cols].head(10))

In [ ]:
# Xuất chi tiết hợp đồng ra Excel (optional)
print("\n5️⃣ Xuất chi tiết hợp đồng ra Excel (optional):\n")

# Uncomment để xuất file
# output_file = f"outputs/Loan_Details_{timestamp}.xlsx"
# df_loan_forecast.to_excel(output_file, index=False)
# print(f"✅ Đã xuất: {output_file}")

print("💡 Tip: Uncomment code trên để xuất chi tiết hợp đồng ra Excel")
print("\n📌 Lưu ý: df_loan_forecast đã có SẴN tất cả thông tin từ df_raw")
print("   Bạn có thể truy cập trực tiếp mà không cần merge thêm!")

---
## 6.5️⃣ ALLOCATION V2: Transition Matrix + Scaling (Nâng cao)

Phương pháp allocation mới:
- Dùng **transition matrix** để assign STATE_FORECAST (thay vì random sampling)
- **Scaling** để match với lifecycle đã calibrated
- Có **backtest** để đánh giá độ chính xác

In [ ]:
# Import allocation_v2
from src.rollrate.allocation_v2_fast import (
    allocate_multi_mob_with_scaling_fast,
    allocate_multi_mob_fast,
    allocate_fast,
)

print("✅ Import allocation_v2_fast thành công")

In [ ]:
# Lấy snapshot mới nhất của loans
latest_cutoff = df_raw['CUTOFF_DATE'].max()
df_loans_latest = df_raw[df_raw['CUTOFF_DATE'] == latest_cutoff].copy()

# Thêm VINTAGE_DATE - dùng parse_date_column để hỗ trợ cả YYYYMM và datetime
df_loans_latest['VINTAGE_DATE'] = parse_date_column(df_loans_latest[CFG['orig_date']])

print(f"📊 Loans snapshot mới nhất:")
print(f"   CUTOFF_DATE: {latest_cutoff}")
print(f"   Số loans: {len(df_loans_latest):,}")

In [ ]:
print("🔨 Allocation V2: Transition Matrix + Scaling (FAST)...")
print("   Target MOBs: 12, 24")
print("   (Thời gian: ~1-2 phút)\n")

df_loan_forecast_v2 = allocate_multi_mob_with_scaling_fast(
    df_loans_latest=df_loans_latest,
    df_lifecycle_final=df_lifecycle_final,
    matrices_by_mob=matrices_by_mob,
    target_mobs=[12, 24],
    parent_fallback=parent_fallback,
    include_del30=True,
    include_del90=True,
    seed=42
)

print(f"\n✅ Allocation V2 hoàn tất: {len(df_loan_forecast_v2):,} loans")

In [ ]:
# So sánh EAD forecast vs current
print("📊 So sánh EAD forecast vs current:\n")

ead_current = df_loan_forecast_v2['EAD_CURRENT'].sum()
print(f"EAD_CURRENT: {ead_current:,.0f}")

for mob in [12, 24]:
    ead_col = f'EAD_FORECAST_MOB{mob}'
    if ead_col in df_loan_forecast_v2.columns:
        ead_forecast = df_loan_forecast_v2[ead_col].sum()
        reduction = (1 - ead_forecast / ead_current) * 100
        print(f"\nMOB {mob}:")
        print(f"   EAD_FORECAST: {ead_forecast:,.0f}")
        print(f"   Reduction: {reduction:.2f}% (dư nợ giảm do thanh toán/prepay/writeoff)")

In [ ]:
# So sánh DEL90 giữa 2 phương pháp
print("📊 So sánh DEL90 giữa 2 phương pháp:\n")

for mob in [12, 24]:
    del90_v1 = df_loan_forecast[f'DEL90_FLAG_MOB{mob}'].mean() * 100
    del90_v2 = df_loan_forecast_v2[f'DEL90_FLAG_MOB{mob}'].mean() * 100
    
    print(f"MOB {mob}:")
    print(f"   V1 (Random sampling): {del90_v1:.2f}%")
    print(f"   V2 (Transition matrix): {del90_v2:.2f}%")
    print(f"   Diff: {del90_v2 - del90_v1:+.2f}%\n")

In [ ]:
# Hiển thị sample kết quả V2
print("📋 Sample kết quả Allocation V2:\n")

display_cols_v2 = [
    'AGREEMENT_ID', 'PRODUCT_TYPE', 'RISK_SCORE',
    'STATE_CURRENT', 'MOB_CURRENT', 'EAD_CURRENT',
    'STATE_FORECAST_MOB12', 'EAD_SCALED_MOB12', 'DEL90_FLAG_MOB12',
    'STATE_FORECAST_MOB24', 'EAD_SCALED_MOB24', 'DEL90_FLAG_MOB24'
]

display_cols_v2 = [c for c in display_cols_v2 if c in df_loan_forecast_v2.columns]
print(df_loan_forecast_v2[display_cols_v2].head(10))

---
## 7️⃣ ANALYSIS & VISUALIZATION

In [ ]:
# DEL90 migration
print("📊 DEL90 Migration Analysis (MOB 12 → 24):\n")

df_migration = compare_del_across_mobs(
    df_multi_mob=df_loan_forecast,
    target_mobs=[12, 24],
    metric="DEL90"
)

In [ ]:
# Pivot table: DEL90% by Product × MOB
df_pivot_del90 = pivot_del_by_product_mob(
    df_multi_mob=df_loan_forecast,
    target_mobs=[12, 24],
    metric="DEL90"
)

print("\n📊 DEL90% by Product × MOB:")
print(df_pivot_del90)

# Visualize
if not df_pivot_del90.empty:
    df_pivot_del90.plot(kind="bar", figsize=(10, 6))
    plt.title("DEL90% by Product (MOB 12 vs MOB 24)")
    plt.xlabel("Product")
    plt.ylabel("DEL90%")
    plt.legend(title="MOB")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

---
## 8️⃣ EXPORT REPORTS

In [ ]:
print("💾 Exporting reports...\n")

# Create output folder
from pathlib import Path
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

# Timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

print("1. Lifecycle (cohort-level)...")

# Actual info for heatmap
actual_info = {}
for (prod, score, vintage), mob_dict in actual_results.items():
    if mob_dict:
        max_mob_actual = max(mob_dict.keys())
        actual_info[(prod, vintage)] = max_mob_actual

# Extend with portfolio
actual_info_all = extend_actual_info_with_portfolio(
    actual_info,
    portfolio_name="PORTFOLIO_ALL"
)

# Export lifecycle
lifecycle_file = output_dir / f"Lifecycle_All_Products_{timestamp}.xlsx"
export_lifecycle_all_products_one_file(
    df_del_prod=df_del_all,
    actual_info=actual_info_all,
    filename=str(lifecycle_file)
)

print(f"   ✅ {lifecycle_file}")

In [ ]:
print("\n2. Loan-level forecast (MOB 12 & 24)...")

loan_file = output_dir / f"Loan_Forecast_MOB12_24_{timestamp}.xlsx"
export_multi_mob_to_excel(
    df_multi_mob=df_loan_forecast,
    filename=str(loan_file),
    target_mobs=[12, 24]
)

print(f"   ✅ {loan_file}")

In [ ]:
print("\n2b. Loan-level forecast V2 (Transition Matrix)...")

# Export V2 (tự động chia sheet nếu > 1M rows)
from src.config import export_loan_forecast_excel
loan_file_v2 = output_dir / f"Loan_Forecast_V2_{timestamp}.xlsx"
export_loan_forecast_excel(
    df_loan_forecast_v2, 
    loan_file_v2, 
    target_mobs=[12, 24],
    include_del_sheets=True
)
print(f"   📌 EAD_FORECAST đã được tính từ lifecycle (luôn <= EAD_CURRENT)")

In [ ]:
print("\n3. Calibration k values...")

# Export k values
k_file = output_dir / f"Calibration_k_values_{timestamp}.xlsx"

with pd.ExcelWriter(k_file, engine="xlsxwriter") as writer:
    # k_raw, k_smooth, k_final
    df_k = pd.DataFrame({
        "MOB": sorted(set(k_raw_by_mob) | set(k_smooth_by_mob) | set(k_final_by_mob)),
    })
    df_k["k_raw"] = df_k["MOB"].map(k_raw_by_mob)
    df_k["k_smooth"] = df_k["MOB"].map(k_smooth_by_mob)
    df_k["k_final"] = df_k["MOB"].map(k_final_by_mob)
    df_k["alpha"] = alpha
    
    df_k.to_excel(writer, sheet_name="k_values", index=False)
    
    # k_raw detail (if available)
    if not k_raw_df.empty:
        k_raw_df.to_excel(writer, sheet_name="k_raw_detail", index=False)

print(f"   ✅ {k_file}")

---
## ✅ SUMMARY

In [ ]:
print("="*60)
print("📊 WORKFLOW SUMMARY")
print("="*60)

print(f"\n1️⃣ Data:")
print(f"   Loans: {df_raw[CFG['loan']].nunique():,}")
print(f"   Products: {df_raw['PRODUCT_TYPE'].nunique()}")
print(f"   Date range: {df_raw[CFG['cutoff']].min()} → {df_raw[CFG['cutoff']].max()}")

print(f"\n2️⃣ Transition Matrices:")
print(f"   Products: {len(matrices_by_mob)}")
print(f"   Total matrices: {sum(len(mob_dict) for mob_dict in matrices_by_mob.values())}")

print(f"\n3️⃣ Lifecycle:")
print(f"   Total rows: {len(df_lifecycle_final):,}")
print(f"   Actual: {(df_lifecycle_final['IS_FORECAST']==0).sum():,}")
print(f"   Forecast: {(df_lifecycle_final['IS_FORECAST']==1).sum():,}")
print(f"   MOB range: {df_lifecycle_final['MOB'].min()} → {df_lifecycle_final['MOB'].max()}")

print(f"\n4️⃣ Calibration:")
print(f"   Alpha: {alpha:.4f}")
print(f"   k_final: {len(k_final_by_mob)} MOBs")
print(f"   k range: {min(k_final_by_mob.values()):.4f} → {max(k_final_by_mob.values()):.4f}")

print(f"\n5️⃣ Loan-level Forecast:")
print(f"   Total loans: {len(df_loan_forecast):,}")
print(f"   DEL90 @ MOB 12: {df_loan_forecast['DEL90_FLAG_MOB12'].sum():,} ({df_loan_forecast['DEL90_FLAG_MOB12'].mean()*100:.2f}%)")
print(f"   DEL90 @ MOB 24: {df_loan_forecast['DEL90_FLAG_MOB24'].sum():,} ({df_loan_forecast['DEL90_FLAG_MOB24'].mean()*100:.2f}%)")

print(f"\n6️⃣ Outputs:")
print(f"   📁 {output_dir}/")
print(f"      - Lifecycle_All_Products_{timestamp}.xlsx")
print(f"      - Loan_Forecast_MOB12_24_{timestamp}.xlsx")
print(f"      - Calibration_k_values_{timestamp}.xlsx")

print("\n" + "="*60)
print("✅ WORKFLOW HOÀN TẤT")
print("="*60)